In [1]:
import utils, img_check
import os

def calculate_pixels(image_path):
    min_height, max_height = img_check.minmax_height(image_path)
    min_width, max_width = img_check.minmax_width(image_path)
    total_pixels = (min_height * min_width * 3)
    return total_pixels

def get_stats_one_partition(folders_path):
    file_paths = utils.get_files_path(folders_path)
    no_of_files = len(file_paths)
    mode = img_check.imagemode(file_paths)
    min_height, max_height = img_check.minmax_height(file_paths)
    min_width, max_width = img_check.minmax_width(file_paths)
    average_mean = img_check.avg_mean(file_paths)
    

def get_stats_mask_partition(folders_path):
    file_paths = utils.get_files_path(folders_path)
    no_of_files = len(file_paths)
    mode = img_check.imagemode(file_paths)
    min_height, max_height = img_check.minmax_height(file_paths)
    min_width, max_width = img_check.minmax_width(file_paths)
    average_mean = img_check.avg_mean(file_paths)
    wb_ratio = img_check.whiteblack_ratio(file_paths)



In [2]:
def wb_dataframe(folders_path):
    file_paths = utils.get_files_path(folders_path)
    wb_dataframe = img_check.wb_ratio_df(file_paths)
    return wb_dataframe


def all_rgb_deviate_pct(image_path, mask_path):
    image_mask_dict = utils.create_image_mask_dict(image_path, mask_path)
    total_pixels = calculate_pixels(image_path)
    all_rgb_dfs = img_check.get_all_rgb_dfs(image_mask_dict)
    avg_rgb_df = img_check.get_avg_rgb_df(image_path, mask_path)
    all_rgb_deviate_pct = img_check.rgb_deviate_pct(all_rgb_dfs, avg_rgb_df, total_pixels)
    return all_rgb_deviate_pct, total_pixels


## Pixel Level stats exploration

In [3]:
def get_pixel_stats(folders_path):
    image_partition_paths = utils.get_files_path(folders_path)
    partition_list_arr = img_check.get_pixels_array(image_partition_paths)
    partition_mean_arr = img_check.get_pixels_mean(partition_list_arr)
    partition_std_arr = img_check.get_pixels_std(partition_list_arr)
    partition_average_image = img_check.get_average_image(partition_mean_arr)
    partition_abs_total_deviation, partition_sum_deviations = img_check.get_deviation(image_partition_paths,  partition_mean_arr, channel = 3)
    partition_min_index, partition_max_index, partition_mean_deviation = img_check.min_max_deviation(partition_sum_deviations, image_partition_paths)
    total_pixels = calculate_pixels(image_partition_paths)
    plot_avg_pixel_deviation = img_check.plot_image(partition_sum_deviations, total_pixels)
    

# Grapher

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
def plot_wb_ratio(wb_dataframe):
    fig, ax =plt.subplots(1,2, figsize=(10,5))
    sns.barplot(x="Index", y="Ratio", data=wb_dataframe, ax=ax[0]).set(title='{prefix}')
    plt.show()

def get_rgb_dataframe(image_path, mask_path):
    rgb_df = img_check.get_rgb_df(image_path, mask_path)
    fig, ax =plt.subplots(1,2, figsize=(25,25))
    sns.lineplot(data=rgb_df, ax=ax[0]).set(title='{prefix}')
    plt.show()

def get_avg_rgb_dataframe(image_path, mask_path):
    rgb_avg_df = img_check.get_avg_rgb_df(image_path, mask_path)
    fig, ax =plt.subplots(1,2, figsize=(25,25))
    sns.lineplot(data=rgb_avg_df, ax=ax[0]).set(title='{prefix}')
    plt.show()

def get_deviation_dataframe(image_path, mask_path):
    rgb_df = img_check.get_rgb_df(image_path, mask_path)
    rgb_avg_df = img_check.get_avg_rgb_df(image_path, mask_path)
    rgb_deviation = rgb_df.subtract(rgb_avg_df).abs()
    fig, ax =plt.subplots(1,2, figsize=(25,25))
    sns.lineplot(data=rgb_deviation, ax=ax[0]).set(title='{prefix}')

def plot_all_rgb_deviate_pct(image_path, mask_path):
    all_rgb_deviate_pct, total_pixels = all_rgb_deviate_pct(image_path, mask_path)
    fig, ax= plt.subplots(1,1, figsize=(10,5))
    sns.lineplot(data=all_rgb_deviate_pct)
    plt.show()  

def plot_histogram(pixel_mean, pixel_std, pixel_distribution, abs_total_deviation, index):
    pixel_mean_hist,bin = np.histogram(pixel_mean.ravel(),256,[0,255])
    pixel_std_hist,bin = np.histogram(pixel_std.ravel(),256,[0,255])
    actual_image,bin = np.histogram(pixel_distribution[index,:,:].ravel(),256,[0,255])
    deviation_image, bin = np.histogram(abs_total_deviation[index,:,:].ravel(),256,[0,255])
    plt.figure(figsize=(15,10))
    plt.xlim([0,255])
    plt.plot(pixel_mean_hist, color = 'red')
    plt.plot(pixel_std_hist, color = 'blue')
    plt.plot(actual_image, color = 'black')
    plt.plot(deviation_image, color = 'green')
    plt.title('histogram of pixel_mean')
    plt.xlabel("Pixel intensity value")
    plt.ylabel("Frequency, # of pixels")
    plt.legend(['Pixel_mean', 'Pixel_std', 
    'actual_image', 'deviation_image'])
    plt.show()

# Compare partitions

In [ ]:
def plot_mean_deviation_across_partitions():
    mean_list = {'type': ['left', 'right', 'all', 'partition_1', 'partition_2',     'partition_3'], 'mean_deviation': [left_mean_deviation, right_mean_deviation, all_mean_deviation, partition_1_deviation, partition_2_deviation, partition_3_deviation]}
    mean_df = pd.DataFrame.from_dict(data=mean_list)

    mean_df
    fig, ax= plt.subplots(1,1, figsize=(10,5))
    sns.barplot(data=mean_df, x = 'type', y = 'mean_deviation')
    plt.show()